In [ ]:
import os, subprocess, sys
gpu = subprocess.run('nvidia-smi', shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).returncode == 0
req = 'requirements-colab-gpu.txt' if gpu else 'requirements-colab-cpu.txt'
subprocess.run(['apt-get','-y','update'], check=True)
subprocess.run(['apt-get','-y','install','ffmpeg'], check=True)
subprocess.run([sys.executable,'-m','pip','install','-r',req], check=True)
if gpu:
    subprocess.run([sys.executable,'-m','pip','install','torch','torchaudio','torchvision','--index-url','https://download.pytorch.org/whl/cu121'], check=True)
else:
    subprocess.run([sys.executable,'-m','pip','install','torch','torchaudio','torchvision','--index-url','https://download.pytorch.org/whl/cpu'], check=True)


In [ ]:
import sys, subprocess, math, wave, array
from pathlib import Path
from ipywidgets import Dropdown, FloatSlider, Text, Button, VBox, Output

def _write_tone(path, freq, duration=5, sr=44100):
    t=[math.sin(2*math.pi*freq*i/sr) for i in range(int(duration*sr))]
    ints=array.array('h',[int(max(-1.0,min(1.0,x))*32767) for x in t])
    path.parent.mkdir(parents=True, exist_ok=True)
    with wave.open(str(path),'wb') as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr); wf.writeframes(ints.tobytes())

def make_demo(directory):
    freqs={'vocals':440,'drums':220,'bass':110,'other':330}
    for name,f in freqs.items():
        _write_tone(directory/f'{name}.wav', f)

inp_widget=Text(value='demo_stems', description='Input')
out_widget=Text(value='demo_output', description='Output')
model_widget=Dropdown(options=['basic','advanced'], value='basic', description='Model')
quality_widget=Dropdown(options=['low','medium','high'], value='medium', description='Quality')
lufs_widget=FloatSlider(value=-14.0, min=-30.0, max=0.0, step=1.0, description='Mix LUFS')
run_btn=Button(description='Run')
preview=Output()

def run(_):
    preview.clear_output()
    inp=Path(inp_widget.value)
    out=Path(out_widget.value)
    if not inp.exists():
        make_demo(inp)
    cmd=[sys.executable, 'scripts/mix_cli.py', str(inp), str(out),
         '--model', model_widget.value,
         '--quality', quality_widget.value,
         '--mix-lufs', str(lufs_widget.value)]
    with preview:
        print('Command:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

run_btn.on_click(run)
VBox([inp_widget, out_widget, model_widget, quality_widget, lufs_widget, run_btn, preview])
